## Urgent Data 
> take 2.0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from doubledot import *
import pandas as pd
from time import sleep
import os
import jmespath as jp

In [ ]:
sf = Salesforce()

In [ ]:
### Clean out old data ?
# for table in Salesforce.model_d.keys():
#     sf.delete_sf_objects(table)

In [ ]:
#| eval: false


# 415645 73012 105765 and 342765 are also good contacts to bring in when you are rea and organization 133156
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167, 415645, 73012, 105765, 342765 ]
pelton_ids = [ 105, 867869, 774081, 380046, 793444, 868245, 785719, 543385 ]
pelton_ids = [ 175861 ]
sf.retrieve_atms_records_by_contactId(pelton_ids)

# number of objects in dictionary
assert len(sf.atms.obj_d) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of objects in dictionary {sf.atms.obj_d.keys()}"
# number of files in directory
assert len(os.listdir(sf.atms.download_dir)) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of files in directory {sf.atms.download_dir}"



Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.
fetch_data_by_contactIds : 175861
we have contact_id 175861 and obj is sales
http://crm-api-neaq.atmsplus.com/api/sales/contact/175861
[{'saleKey': '6450070', 'saleAmount': '70.0000', 'paymentAmount': '70.0000', 'saleDate': '2023-05-29T12:55:40.85', 'active': True, 'terminalKey': 78, 'ticketCount': 2, 'eventDate': '2023-05-29T13:00:00', 'booking': {'bookingId': 1901565, 'bookingContactKey': 2519260, 'bookingContactType': 'Primary', 'contactKey': 175861, 'contactIndividualKey': 169321, 'contactOrganizationKey': 0, 'displayName': 'Cheung, David', 'firstName': 'David', 'lastName': 'Cheung', 'email': 'david.cheung@vantixsystems.com', 'phone': '7804210499'}, 'saleComment': None, 'saleDetails': [{'saleDetailId': 13544390, 'saleId': 6450070, 'itemKey': 55, 'scheduleKey': 255449, 'rateKey': 5, 'categoryKey': 4, 'itemQuantity': 2, 'pricingPriceKey': 10071, 'itemPrice': 34.0, 'itemTotal': 68.0, 'couponTotal': 0.0, 

AssertionError: wrong number of objects in dictionary dict_keys(['contacts', 'sales', 'memberships'])

In [ ]:
search_s = "[].{LastName: lastName,\
        FirstName: firstName,\
        MailingPostalCode: addresses[0].postalZipCode,\
        MailingCity: addresses[0].city,\
        MailingStreet: addresses[0].line1, \
        MailingCountry: addresses[0].country, \
        Phone: phones[?phoneType == 'Business'].phoneNumber | [0],\
        Email: emails[0].address,\
        contactId__c: contactId}"

dict_l = jp.search(search_s, sf.atms.obj_d['contacts'])
df = pd.DataFrame(dict_l)
df

,LastName,FirstName,MailingPostalCode,MailingCity,MailingStreet,MailingCountry,Phone,Email,contactId__c
0,Cheung,David,T5K2T3,Edmonton,10119 97 Ave,Canada,7804210499,david.cheung@vantixsystems.com,175861


## Primary Test Method

In [ ]:
####### PRIMARY TEST METHOD ########
sf.clean_upload_dir()
# sf.upload_csv_to_sf(clean_all=False, obj_l=['MembershipTermMemberRel__c'], generate_upload_files=True)
# sf.upload_csv_to_sf(clean_all=False, generate_upload_files=True)
sf.atms.obj_d= {}
sf.upload_csv_to_sf(clean_all=True, generate_upload_files=True)

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.
Deleting Contact objects from Salesforce
Retrieving Object Ids for Contact from Salesforce
total number of objects =  15
In Salesforce.delete_sf_objects: Deleting 15 Contact objects using /Users/josephmann/Documents/Github/doubledot/sf_download/Contact.csv
execute_job
Created job 7508Y00000nJIQuQAO for Contact with operation delete
Uploading job 7508Y00000nJIQuQAO of object Contact
Uploaded job 7508Y00000nJIQuQAO of object Contact with status 201
response.text : 
 : 
close_job response.text :
 {"id":"7508Y00000nJIQuQAO","operation":"delete","object":"Contact","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-06-01T18:03:54.000+0000","systemModstamp":"2023-06-01T18:03:54.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0}
pre loop job status: {'id': '7508Y00000nJIQuQAO', 'operation': 'delete', 'object': 'Contact', 'createdById': '0058Y00000CIEn9QAH', 

AssertionError: file_path_s /Users/josephmann/Documents/Github/doubledot/sf_upload/Contact.csv does not exist in process_contacts

In [ ]:
sf.clean_upload_dir()
sf.process_memberships()
sf.process_saleDetails()
sf.process_term_member_rel()

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.
write_jmespath_to_csv Membership__c
write_jmespath_to_csv [{'membershipId__c': 295573, 'memberSince__c': '2019-05-20T23:02:11.007', 'updateDate__c': '2023-01-04T15:09:22.887'}, {'membershipId__c': 295573, 'memberSince__c': '2019-05-20T23:02:11.007', 'updateDate__c': '2023-01-04T15:09:22.887'}, {'membershipId__c': 13, 'memberSince__c': '2009-09-25T00:00:00', 'updateDate__c': '2022-12-30T14:52:16.68'}, {'membershipId__c': 13, 'memberSince__c': '2009-09-25T00:00:00', 'updateDate__c': '2022-12-30T14:52:16.68'}, {'membershipId__c': 302188, 'memberSince__c': '2020-09-18T11:56:22.1', 'updateDate__c': '2022-12-09T16:04:34.39'}]
write_jmespath_to_csv for Membership__c dtypes membershipId__c    object
memberSince__c     object
updateDate__c      object
dtype: object
write_jmespath_to_csv MembershipTerm__c
write_jmespath_to_csv [{'membershipTermId__c': '416780', 'membershipKey__r_1_membershipId__c': 295573, 'effecti

In [ ]:
pd.DataFrame()

In [ ]:
# sf.delete_sf_objects('MembershipTermMemberRel__c')
# assert os.path.exists(os.path.join(Salesforce.class_upload_dir,'MembershipTermMemberRel__c.csv')), 'upload file MembershipTermMemberRel__c does not exist'
sf.execute_job('MembershipTermMemberRel__c', 'insert', use_ATMS_data=True) 


execute_job
Created job 7508Y00000nJ2oMQAS for MembershipTermMemberRel__c with operation insert
Uploading job 7508Y00000nJ2oMQAS of object MembershipTermMemberRel__c
Uploaded job 7508Y00000nJ2oMQAS of object MembershipTermMemberRel__c with status 201
response.text : 
 : 
close_job response.text :
 {"id":"7508Y00000nJ2oMQAS","operation":"insert","object":"MembershipTermMemberRel__c","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-31T03:12:06.000+0000","systemModstamp":"2023-05-31T03:12:06.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0}
pre loop job status: {'id': '7508Y00000nJ2oMQAS', 'operation': 'insert', 'object': 'MembershipTermMemberRel__c', 'createdById': '0058Y00000CIEn9QAH', 'createdDate': '2023-05-31T03:12:06.000+0000', 'systemModstamp': '2023-05-31T03:12:09.000+0000', 'state': 'InProgress', 'concurrencyMode': 'Parallel', 'contentType': 'CSV', 'apiVersion': 57.0, 'jobType': 'V2Ingest', 'lineEnding': 'LF', 'columnD

## use pandas to create outer join of Members and Terms


In [ ]:

members_df = pd.read_csv('sf_upload/MembershipMember__c.csv', sep='\t')
terms_df = pd.read_csv('sf_upload/MembershipTerm__c.csv', sep='\t')
join_df = members_df.merge(terms_df, left_on='membershipTermKey__r.membershipTermId__c', right_on='membershipTermId__c', how='outer')    
# terms_df\
join_df[['membershipMemberId__c', 'membershipTermId__c']].to_csv('sf_upload/MembershipTermMemberRel__c.csv', sep='\t', index=False, index_label='Name')


In [ ]:
d_l  = sf.process_saleDetails()

NameError: name 'sf' is not defined

## Problems ...


In [ ]:
#|eval: false

# why does saleId__c 889832 not have any tickets or saleDetail pointing at it? or me: 82192 323674 (I am 73012)
# why does this point from saleId and not saleKey?
from doubledot.crema_sf import fields_pointing_to_foreign_key, get_pointing_foreign_key_values,reduce_to_referenced_rows

_df = df_d['Sale__c']
print(_df.loc[_df['saleId__c'] == 323674, :])

print(get_pointing_foreign_key_values('saleId__c', df_d))
pointing = get_pointing_foreign_key_values('saleId__c', df_d)
print(len(pointing))

print(_df['saleId__c'].unique())
externals = _df['saleId__c'].unique()

pointing.intersection(externals)
      

In [ ]:
reduce_to_referenced_rows(df_d)


In [ ]:
_d['Ticket__c']['saleKey__r.saleId__c']

In [ ]:
int("1344.")

ValueError: invalid literal for int() with base 10: '1344.'